# 1. Импортируем библиотеки

In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import random


import scipy
from sklearn.model_selection import train_test_split

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k

from lightfm import LightFM
from lightfm.evaluation import precision_at_k

import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm.notebook import tqdm as  tqdm_notebook

C:\Users\derev\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# 2. Импорт данных

### 2.1. Словарь с id и наименованиеи лекарств

In [2]:
with open('Product_dict.pkl', mode='rb') as f:
    product_dict = pickle.load(f)

In [3]:
product_dict

{'168308': '(197312) Пакет-майка 25см х 45см 906',
 '134832': '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '101384': '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404',
 '168570': '(197309) Пакет 28см х 50см 906',
 '146960': '(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '128700': '(51806) Ранитидин тб 150мг уп N20 595',
 '52539': '(19416) Платки носовые "Зева Кидс" №10 721',
 '94270': '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1',
 '77426': '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759',
 '92475': '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575',
 '158643': '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762',
 '158600': '(181232) PL Шприц одноразовый 3-комп. 5мл №1 762',
 '106230': '(43344) Аллапинин тб 25мг N30 738',
 '110629': '(57733) Корвалол капли д/приема внутрь 25мл 738',
 '47251': '(121286) PL Контейнер д/биопроб стер. 60мл с крыш и ложкой в инд.уп 711',
 '39811': '(116543) PL Салфетка антисептическая спи

### 2.2. Чековые данные

In [4]:
cheque = pd.read_csv('чековые данные.csv')

C:\Users\derev\AppData\Local\Temp\ipykernel_1740\2904546257.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  cheque = pd.read_csv('чековые данные.csv')


In [5]:
# Проверим длину датасета с чековыми данными
print('Длина датасета с чековыми данными: ', len(cheque), ' строк')

Длина датасета с чековыми данными:  20000003  строк


In [6]:
# Для работы выделим только 100 тыс. верхних строк
cheque_data = cheque.head(50000).copy()
cheque_data.head(5)

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


# 3. Предобработка данных

In [7]:
# Посмотрим информацию о данных
cheque_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sale_date_date           50000 non-null  object 
 1   contact_id               50000 non-null  object 
 2   shop_id                  50000 non-null  float64
 3   product_id               50000 non-null  float64
 4   name                     49999 non-null  object 
 5   product_sub_category_id  50000 non-null  float64
 6   product_category_id      50000 non-null  float64
 7   brand_id                 50000 non-null  float64
 8   quantity                 50000 non-null  object 
dtypes: float64(5), object(4)
memory usage: 3.4+ MB


In [8]:
# Проверим пропуски
cheque_data.isnull().sum()

sale_date_date             0
contact_id                 0
shop_id                    0
product_id                 0
name                       1
product_sub_category_id    0
product_category_id        0
brand_id                   0
quantity                   0
dtype: int64

In [9]:
# Проверим данные
cheque_data.describe()

,shop_id,product_id,product_sub_category_id,product_category_id,brand_id
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,813.353600,106156.647860,598.547180,137.357060,541.106860
std,612.649499,55058.385012,135.890837,31.213633,1040.130691
min,41.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,312.000000,65610.000000,456.000000,107.000000,-1.000000
50%,598.000000,104256.000000,627.000000,142.000000,-1.000000
75%,1470.000000,140688.000000,705.000000,162.000000,876.000000
max,2166.000000,363111.000000,935.000000,208.000000,5475.000000


In [10]:
# Проверим почему есть пропуски в столбце name
name_lst = []
for i, j in zip(cheque_data.name, cheque_data.product_id):
    if pd.isna(i):
        if j not in name_lst:
            name_lst.append(j)
            
print(name_lst)

[-1.0]


In [11]:
# Проверим можно ли заполнить пропуски из словаря с лекарствами
for i in name_lst:
    if pd.isna(i):
        print('NaN')
    elif str(int(i)) in product_dict.keys():
        print(i, ": ", product_dict[str(int(i))])
    else:
        print(i, ": Not found")

-1.0 :  N/A -1


In [12]:
# В словаре с лекарствами наименования для id из пропусков не оказалось. Просто удалим пропуски
cheque_data = cheque_data.dropna()

In [13]:
# Переименуем столбец с датой
cheque_data = cheque_data.rename(columns={'sale_date_date': 'date'})

In [14]:
# Преобразуем данные в столбцax contact_id и product_id в int
cheque_data[['contact_id', 'product_id']] = cheque_data[['contact_id', 'product_id']].astype('int')

In [15]:
# Отделим id лекарства от названия лекарства
cheque_data['drug_id'] = cheque_data.name.apply(lambda x: re.findall(r'[^()]+', x)[0])
cheque_data['name'] = cheque_data.name.apply(lambda x: re.sub(r'^[(]\w+[)]', '', x))
cheque_data.head(5)

,date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity,drug_id
0,2018-12-07,1260627,1455.0,168308,Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00",197312
1,2018-12-07,198287,279.0,134832,Перекись водорода р-р наружн. 3% фл.полимерн....,404.0,93.0,-1.0,"1,00",62448
2,2018-12-07,2418385,848.0,101384,Салициловая кислота р-р спирт 2% фл 40мл N1,404.0,93.0,-1.0,"1,00",72183
3,2018-12-07,1285774,1511.0,168570,Пакет 28см х 50см,906.0,205.0,-1.0,"1,00",197309
4,2018-12-07,1810323,1501.0,168319,Пакет 30см х 60см,906.0,205.0,-1.0,"1,00",197310


In [16]:
# Проверим уникальные значения в столбце quantity
cheque_data.quantity.value_counts()

1,00     19637
2,00     17266
3,00      3831
5,00      1961
4,00      1564
         ...  
38,00        1
0,625        1
0,173        1
0,004        1
0,168        1
Name: quantity, Length: 216, dtype: int64

In [17]:
# Преобразуем данные в столбце quantity в float
cheque_data['quantity'] = cheque_data['quantity'].str.replace(",", ".")
#cheque_data['quantity'] = cheque_data['quantity'].apply(lambda x: float(x))
cheque_data['quantity'] = pd.to_numeric(cheque_data.quantity)

In [18]:
#cheque_data.select_dtypes(include=['float64'])

In [19]:
# Создадим колонку объединив информацию по дате покупки и id покупателя
cheque_data['contact_id_date'] = cheque_data['date'].astype(str) + " " + cheque_data['contact_id'].astype(str)

In [20]:
# Проставим единицу для каждого пользователя в каждой строке с покупкой лекарства(товара)
cheque_data['buy'] = (cheque_data['quantity'] > 0).apply(int)

# 4. ALS и LightFM

In [21]:
# Создадим отдельный датасет с нужными нам ячейками
cheque_df = cheque_data[['date', 'contact_id',
                         'product_id', 'buy']].reset_index(drop=True)
cheque_df = cheque_df.rename(columns={'contact_id':'user_id', 'product_id':'item_id'})
# Train & Test splitting
train, test = train_test_split(cheque_df, test_size=0.2, shuffle=False)

# Pivoting TRAIN
train_pivot = pd.pivot_table(train,
                             #index=['date', 'contact_id'],
                             index='user_id',
                             columns='item_id',
                             values='buy')

# Pivoting TEST
test_pivot = pd.pivot_table(test,
                            #index=['date', 'contact_id'],
                            index='user_id',
                            columns='item_id',
                            values='buy')

In [22]:
# Проверим форму матриц
print(train_pivot.shape)
print(test_pivot.shape)

(23563, 5965)
(5467, 3586)


In [23]:
# Создадим сводную таблицу из таблицы cheque_df, заполнив её ячейки нулями. 
# Для тех лекарств(товаров), которые пользователь купил, будут равны 0, для остальных — пропуску.
shell = pd.pivot_table(
    cheque_df, 
    #index=['date', 'contact_id'], 
    index='user_id',
    columns="item_id", 
    values="buy", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,8106,8111,8118,8119,8128,17906,17914,17939,18040,18075,...,359480,359508,359678,359708,359922,359964,360106,360149,362464,363111
user_id,,,,,,,,,,,,,,,,,,,,,
457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Получим тренировочную и тестовую user-item таблицы, сложив таблицу shell с соответствующими таблицами train_pivot и test_pivot.
* 0 — если пользователь не покупал данное лекарство(товар);
* 1 — если пользователь окупал данное лекарство(товар).

In [24]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = train_pivot.fillna(0)
test_pivot = test_pivot.fillna(0)

print(train_pivot.shape)
print(test_pivot.shape)

train_pivot.head()

(26059, 6816)
(26059, 6816)


item_id,8106,8111,8118,8119,8128,17906,17914,17939,18040,18075,...,359480,359508,359678,359708,359922,359964,360106,360149,362464,363111
user_id,,,,,,,,,,,,,,,,,,,,,
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Получим разреженные матрицы, используется функция csr_matrix() из модуля sparse библиотеки scipy:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

In [26]:
# Обучим модель на AlternatingLeastSquares
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

C:\Users\derev\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
# Сформируем рекомендации и его вектор оценок лекарств(товаров).
unique_items = np.array(train_pivot.columns)
user_id = 22
recomendations_ids, scores = model.recommend(user_id, train_pivot_sparse[user_id])
recomendations = unique_items[recomendations_ids]
print('Recomendations ids: {}'.format(recomendations_ids))
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations ids: [4864  605 6487 3603 3758 3950  909 6087  814 6049]
Recomendations for user 22: [134832  39811 168570 106860 110629 114395  51228 158586  47251 156491]


In [28]:
# Определим качество модели на всей тестовой выборке, рассчитав precision для топ 10-рекомендуемых лекарствах(товарах) 
# с помощью функции mean_average_precision_at_k()
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

  0%|          | 0/5467 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.005


Итак, наша метрика равна  0.005. То есть из 10 рекомендуемых лекарств(товаров) пользователи в среднем покупают 0.5 % из них.

In [29]:
# Обучим модель на LightFM
model_lfm = LightFM(no_components=10, loss='warp', random_state=42)
model_lfm.fit(train_pivot_sparse, epochs=30)

In [30]:
# Получим рекомендации, умножив веса на -1 и отсортируем их индексы по возрастанию веса.
item_ids = np.arange(0, train_pivot_sparse.shape[1])
list_pred = model_lfm.predict(user_id, item_ids)
recomendations_ids = np.argsort(-list_pred)[:10]
recomendations = unique_items[recomendations_ids]
print('Recomendations for user {}: {}'.format(user_id, recomendations))

Recomendations for user 22: [ 78665  41096 127362  97117 163329  79735 104368  50859 114395 349828]


In [31]:
# Рассчитаем средний precision для топ 10 рекомендуемых лекарств(товаров) по всей тестовой выборке
map_at10 = precision_at_k(model_lfm, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

Mean Average Precision at 10: 0.004


Метрика ухудшилась и равна  0.004. То есть из 10 рекомендуемых лекарств(товаров) пользователи в среднем покупают 0.4 % из них.

# 5. ContextualRecomendations

In [32]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [33]:
len(product_dict)

30418

In [34]:
# Создадим датасет из словаря лекарств
df_drugs = pd.DataFrame.from_dict(product_dict, orient='index', columns=['name']).reset_index()
df_drugs = df_drugs.rename(columns={'index': 'product_id'})
df_drugs

,product_id,name
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"
...,...,...
30413,193603,(110044169) название -1
30414,193484,(110050486) название -1
30415,192459,(110020427) название -1
30416,193510,(110047818) название -1


In [35]:
# Отделим id лекарства от названия лекарства
df_drugs['text'] = df_drugs.name.apply(lambda x: re.findall(r"(.*).{3}$", re.sub(r'^[(]\w+[)]', '', x))[0])
df_drugs['drug_id'] = df_drugs.name.apply(lambda x: re.findall(r'[^()]+', x)[0])

df_drugs['product_sub_category_id'] = df_drugs.name.apply(lambda x: re.findall(r".{3}$", re.sub(r'^[(]\w+[)]', '', x))[0])

df_drugs['name'] = df_drugs.text


# Удалим дубликаты
df_drugs = df_drugs.drop_duplicates(subset=['text']).reset_index(drop=True)
df_drugs

,product_id,name,text,drug_id,product_sub_category_id
0,168308,Пакет-майка 25см х 45см,Пакет-майка 25см х 45см,197312,906
1,134832,Перекись водорода р-р наружн. 3% фл.полимерн....,Перекись водорода р-р наружн. 3% фл.полимерн....,62448,404
2,101384,Салициловая кислота р-р спирт 2% фл 40мл N1,Салициловая кислота р-р спирт 2% фл 40мл N1,72183,404
3,168570,Пакет 28см х 50см,Пакет 28см х 50см,197309,906
4,146960,"Пакет ""Аптека Озерки"" 28 х 35см","Пакет ""Аптека Озерки"" 28 х 35см",111023,906
...,...,...,...,...,...
27407,153665,Очки корригирующие Кемнер Оптикс глянцевые пл...,Очки корригирующие Кемнер Оптикс глянцевые пл...,121891,663
27408,164193,Лактулоза форте пор.15мл №10,Лактулоза форте пор.15мл №10,190054,597
27409,153013,Курапрокс Набор (зубная паста Вайт Из Блэк 90...,Курапрокс Набор (зубная паста Вайт Из Блэк 90...,121465,528
27410,59719,Лиерак Премиум Набор Бархат Новогодний (крем ...,Лиерак Премиум Набор Бархат Новогодний (крем ...,117974,679


In [36]:
# Преобразуем тип text данных в признаке на str
df_drugs['text'] = df_drugs['text'].apply(lambda x: preprocess_txt(str(x)))
df_drugs.head(5)

,product_id,name,text,drug_id,product_sub_category_id
0,168308,Пакет-майка 25см х 45см,"[пакетмайк, 25см, х, 45см]",197312,906
1,134832,Перекись водорода р-р наружн. 3% фл.полимерн....,"[перекись, водород, рр, наружн, 3, флполимерна...",62448,404
2,101384,Салициловая кислота р-р спирт 2% фл 40мл N1,"[салициловый, кислота, рр, спирт, 2, фл, 40мл,...",72183,404
3,168570,Пакет 28см х 50см,"[пакет, 28см, х, 50см]",197309,906
4,146960,"Пакет ""Аптека Озерки"" 28 х 35см","[пакет, аптека, озерко, 28, х, 35см]",111023,906


In [37]:
# Загрузим текстовые описания товаров и преобразуем к нормализованному виду
# cheque_data['text'] = cheque_data['name']
# cheque_data['text'] = cheque_data['text'].apply(lambda x: preprocess_txt(str(x)))
sentences = [i for i in df_drugs['text'] if len(i) > 2]

# Обучим модель fasttext на текстах
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [38]:
# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_drug = annoy.AnnoyIndex(20 ,'angular')

index_map_drug = {}
counter = 0

for i in tqdm_notebook(range(len(df_drugs))):
    n_ft = 0
    index_map_drug[counter] = (df_drugs["name"].iloc[i], df_drugs["product_id"].iloc[i])
    #print(counter, index_map_drug[counter])
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in df_drugs["text"].iloc[i]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_drug.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_drug.build(10)

  0%|          | 0/27412 [00:00<?, ?it/s]

True

In [39]:
def recommend(item_id):
    # Получим тайтл айтема по идентификатору
    title = cheque_data[cheque_data['product_id']==item_id]['name'].values[0]
    title = preprocess_txt(title)
    
    vector_ft = np.zeros(20)
    n_ft = 0
    # Каждое слово обернем в эмбеддинг
    for word in title:
        #print(word)
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            #print(vector_ft)
            n_ft += 1
            #print(n_ft)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
        #print('vector_ft', vector_ft)
    # Найдем ближайшие по тайтлу
    rec_items = ft_index_drug.get_nns_by_vector(vector_ft, 6)
    # print(rec_items)
    return [index_map_drug[i][0] for i in rec_items if int(index_map_drug[i][1])!= item_id]

####  Получилось так, что для входного параметра в самой первой строке он же и советуется, сделал "костыльное" решение, можно ли как-то обойти такую рекомендация? Буду благодарен за ответ.

In [42]:
# Проверим результат
product_id = input('Ввведите код товара: ')
product_id = int(product_id)
product_name = cheque_data['name'].loc[cheque_data['product_id']==product_id].unique()[0]
result = recommend(product_id)
print("Рекомендации к товару '{0}': \n{1}".format(product_name, result))

Ввведите код товара: 146960
Рекомендации к товару ' Пакет "Аптека Озерки" 28 х 35см': 
[' Пакет Радуга Кидс п/э 29 х 40см ', ' Пакет "Аптека Озерки" 17 х 34,5см ПНД ', ' Пакет "Аптека Озерки" 24 х 45см ПНД ', ' Пеленка Еврон софт супер однораз. 60см х 90см впитыв.№30', ' Пакет "Аптека Озерки" 38 х 50см ']


# 6. MultiStageRecs

In [43]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("en")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [44]:
# Подготовим кандидатогенератор, который будет отдавать лекарства(товары) похожие по текстовому описанию на те, 
# которые купил посетитель аптеки
# Обучим Fasttext и заэмбедим лекарства(товары)
sentences = [i for i in df_drugs['text'] if len(i) > 2]


# Обучим модель fasttext на текстах
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_drug = annoy.AnnoyIndex(20 ,'angular')

index_map_drug = {}
reverse_index_map = {}
counter = 0

for i in tqdm_notebook(range(len(df_drugs))):
    n_ft = 0
    index_map_drug[counter] = (df_drugs["name"].iloc[i], df_drugs["product_id"].iloc[i])
    #print(counter, index_map_drug[counter])
    reverse_index_map[int(df_drugs.loc[i, "product_id"])] = counter
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in df_drugs["text"].iloc[i]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_drug.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_drug.build(10)

  0%|          | 0/27412 [00:00<?, ?it/s]

True

In [45]:
# Создадим метод, который по id лекарства(товара) будет возвращать похожие на него по описанию лекарства(товары)
def recommend(product_id, num_sampled=10):
    if product_id not in reverse_index_map:
        return []
    return ft_index_drug.get_nns_by_item(reverse_index_map[product_id], num_sampled)

In [46]:
# Сделаем сразу рандомный кандидатогенератор (он нам пригодится для кандидатогенерации по лекарствам(товарам))
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=False)

In [47]:
# Сделаем кандидатогенератор по последним купленным лекарствам(товарам)
cheque_dict = {}
drug_dict = {}

for i in range(len(cheque_df)):
    item_id = int(cheque_df.loc[i, "item_id"])
    cheque_dict[cheque_df.loc[i, 'user_id']] = item_id
    #print(cheque_dict)
    #for drug in item_id:
    if item_id not in drug_dict:
        drug_dict[item_id] = []
    drug_dict[item_id].append(cheque_df.loc[i, 'user_id'])

In [48]:
# Создадим метод, который по последнему купленному лекарству(товару) будет возвращать набор кандидатов
drug_unique = df_drugs['product_id'].unique()

def candidate_generator(product_id):
    candidates_ft = []
    if product_id is not None:
        candidates_ft = list(recommend(product_id))
    candidates_random = list(sample_random(drug_unique))

    candidates_drug = []
    for drug in cheque_dict.get(product_id, []):
        candidates_drug += list(sample_random(drug_dict[drug], 5))
    # Отфильтруем входное лекарство(товар) (он мог попасть в кандидаты) и уберем дубликаты
    return set([int(i) for i in candidates_ft + candidates_random + candidates_drug if i!= product_id])

In [49]:
# Проверим кандидатогенератор на рандомном product_id
print('Кандидаты для product_id = {0}: {1}'.format(np.random.choice(cheque_df.item_id), 
                                                   candidate_generator(np.random.choice(cheque_df.item_id)))) 

Кандидаты для product_id = 94105: {324104, 166793, 154126, 162964, 6422, 8343, 20637, 221983, 43, 146603, 80303, 213295, 1089, 337, 1376, 51943, 2162, 153204, 13301, 1402}


In [50]:
# Подготовим наши ранжирующие факторизационные машины
train_pivot = (cheque_df.drop_duplicates(subset='user_id').pivot(index="user_id",
                                                                 columns="item_id",
                                                                 values="buy")).fillna(0)
train_pivot.reset_index(inplace=True, drop=False)
train_pivot

item_id,user_id,8106,8111,8118,8119,8128,17906,17914,18040,18075,...,358817,359333,359398,359480,359508,359922,359964,360106,360149,362464
0,457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26054,2746216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26055,2746243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26056,2746247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26057,2746248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Создадим словарь с соотношением индекса и реального значения user_id 
user_id_dict = train_pivot.user_id.to_dict()
user_id_dict

{0: 457,
 1: 1406,
 2: 1431,
 3: 1445,
 4: 1471,
 5: 1725,
 6: 1791,
 7: 1858,
 8: 1887,
 9: 2018,
 10: 2202,
 11: 2281,
 12: 2360,
 13: 2362,
 14: 2392,
 15: 2497,
 16: 2727,
 17: 2836,
 18: 2946,
 19: 3054,
 20: 3119,
 21: 3145,
 22: 3162,
 23: 3397,
 24: 3554,
 25: 3700,
 26: 3842,
 27: 3879,
 28: 3973,
 29: 3999,
 30: 4038,
 31: 4180,
 32: 4262,
 33: 4439,
 34: 4597,
 35: 4665,
 36: 4695,
 37: 5032,
 38: 5147,
 39: 5296,
 40: 5313,
 41: 5314,
 42: 5489,
 43: 5647,
 44: 5677,
 45: 5685,
 46: 5713,
 47: 5828,
 48: 5911,
 49: 6034,
 50: 6057,
 51: 6101,
 52: 6285,
 53: 6311,
 54: 6484,
 55: 6510,
 56: 6557,
 57: 6775,
 58: 6931,
 59: 7005,
 60: 7057,
 61: 7239,
 62: 7409,
 63: 7453,
 64: 7606,
 65: 7635,
 66: 7705,
 67: 7927,
 68: 7937,
 69: 7938,
 70: 8051,
 71: 8060,
 72: 8078,
 73: 8225,
 74: 8290,
 75: 8465,
 76: 8507,
 77: 8556,
 78: 8761,
 79: 8807,
 80: 9102,
 81: 9174,
 82: 9175,
 83: 9240,
 84: 9406,
 85: 9415,
 86: 9455,
 87: 9518,
 88: 9648,
 89: 9657,
 90: 9868,
 91: 9871,

In [52]:
def get_key(dictionary, value):
    """
    Функция получения ключа словаря по значению.
    """
    for k, v in dictionary.items():
        if v == value:
            return k

In [53]:
# Создадим массив с унмкальными item_id
unique_items = np.array(train_pivot.columns[1:])
unique_items

array([8106, 8111, 8118, ..., 360106, 360149, 362464], dtype=object)

In [54]:
# Получим разреженную матрицу используя функцию csr_matrix() из модуля sparse библиотеки scipy и Обучим модель на LightFM
train_pivot = scipy.sparse.csr_matrix(train_pivot)

model = LightFM(loss='warp')
model.fit(train_pivot, epochs=30)

In [55]:
# Посмотрим, насколько статичные рекомендации будут хороши: 
# возьмем последние купленные лекарства(товары) для каждого пользователя в тренировочном сете
item_ids = set(train['item_id'].unique())
known_buy = train[train['buy'] > 0].groupby('user_id')['item_id'].unique().to_dict()

In [56]:
# Получаем последние купленные лекарства(товары)
last_buy = train[train['buy'] > 0].drop_duplicates(['user_id'], keep="last")[["user_id", "item_id"]].to_dict()
test_grouped = test[test['buy'] > 0].groupby('user_id')['item_id'].unique().to_dict()

In [60]:
# Проверим результат выдачи рекомендация
map_at10 = 0

for user_id, purchases in tqdm_notebook(test_grouped.items()):
    #print(user_id)
    values = set(purchases)
    train_buy = set(known_buy.get(user_id, []))
    # print(train_buy)
    candidates = list(candidate_generator(last_buy.get(user_id, None)))
    # print(candidates)
    
    # Факторизационные машины не умеют работать с айтемами, которых они не видели
    # Поэтому надо отфильтровать кандидатов
    candidates = [i for i in candidates if i in item_ids]
    # print(candidates)
    
    # Если мы не смогли ничего отскорить, то считаем, что пользователь просто получил нашу выдачу в рандомном порядке
    recommendations = candidates[:min(10, len(candidates))]
    if len(candidates) != 0:
        recommendations = sorted(list(zip(item_ids, unique_items[np.argsort(-model.predict(get_key(user_id_dict, user_id), np.arange(0, train_pivot.shape[1])))[:10]])), key=lambda x: x[1], reverse=True)
        recommendations = set([i[0] for i in recommendations if i not in train_buy][:10])
    map_at10 += len(values.intersection(recommendations)) / 10
    # print(map_at10)
map_at10 = map_at10 / len(test_grouped)
print('Mean Average Precision at 10: {:.5f}'.format(map_at10))

  0%|          | 0/5467 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.00011


Качество просело, это объясняется тем, что мы начали рекомендовать из всего пула лукарств. Низкие метрики получились скорее всего из-за того, что ввиду ограничения мощности железа, пришлось сократить исходную выборку параметров до 50000.